In [1]:
import timm
import glob
import time
import logging
import torch
import torch.nn as nn
import torch.nn.parallel
from torch.autograd import Variable
from collections import OrderedDict
from timm.data import Dataset, create_loader, resolve_data_config,  FastCollateMixup, mixup_batch, AugMixDataset
from timm.models import create_model, resume_checkpoint, convert_splitbn_model, apply_test_time_pool
from timm.utils import *
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy, JsdCrossEntropy
from timm.optim import create_optimizer
from timm.scheduler import create_scheduler
from munch import Munch
import yaml
import sys
from resnet_generator import Generator, Critic
from datetime import datetime
import numpy as np
import torch.distributed as dist
import torch.nn.functional as F
from transformer import Attention
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
from crd.criterion import CRDLoss
from dataset.imagenet import get_dataloader_sample
from timm.data import transforms_factory

In [2]:
# from apex import amp
# from apex.parallel import DistributedDataParallel as DDP
# from apex.parallel import convert_syncbn_model
has_apex = False

In [3]:
# GPU Device
gpu_id = '0,1,2'
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device " , use_cuda)

GPU device  True


In [4]:
torch.backends.cudnn.benchmark = True

In [5]:
with open('config/train.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
args = Munch(config)
args.prefetcher = not args.no_prefetcher
args.distributed = False
args.device = 'cuda'
args.world_size = 3
args.rank = 0
logging.info('Training with a single process on %d GPUs.' % args.num_gpu)

INFO:root:Training with a single process on 3 GPUs.


In [6]:
if args.distributed:
    args.num_gpu = 1
    args.device = 'cuda:%d' % args.local_rank
    torch.cuda.set_device(args.local_rank)
    torch.distributed.init_process_group(backend='nccl', init_method='env://', rank=args.rank, world_size=args.world_size)
    args.world_size = torch.distributed.get_world_size()
    args.rank = torch.distributed.get_rank()
    assert args.rank >= 0

In [7]:
torch.manual_seed(args.seed + args.rank)

In [8]:
model_ns = timm.create_model('tf_efficientnet_b1_ns', pretrained=True)
model_ns = model_ns.cuda()
model_ns = torch.nn.DataParallel(model_ns)
model_ns = model_ns.eval()

In [9]:
# fc_name = 'module.classifier.'
# teacher_model_weights = {}
# for name, param in model_ns.named_parameters():
#     teacher_model_weights[name] = param.detach()

In [10]:
model_raw = timm.create_model('tf_efficientnet_b1', pretrained=True)
model_raw = model_raw.cuda()
model_raw = torch.nn.DataParallel(model_raw)
if args.resume:
    print('raw load')
    model_raw.load_state_dict(torch.load('raw').state_dict())

In [11]:
model_g = Generator(args, img_size=args.img_size, max_conv_dim=256)
model_g = model_g.cuda()

In [12]:
model_critic = Critic(args)
model_critic = model_critic.cuda()
model_critic = torch.nn.DataParallel(model_critic)
if args.resume:
    print('critic load')
    model_critic.load_state_dict(torch.load('critic').state_dict())

In [13]:
use_amp = False
if has_apex and args.amp:
    model, optimizer = amp.initialize(model, optimizer, opt_level='O1')
    model_raw =  amp.initialize(model_raw)
    model_ns = amp.initialize(model_ns)
    use_amp = True
if args.local_rank == 0:
    logging.info('NVIDIA APEX {}. AMP {}.'.format(
        'installed' if has_apex else 'not installed', 'on' if use_amp else 'off'))

INFO:root:NVIDIA APEX not installed. AMP off.


In [14]:
train_dir = '/home/data/imagenet/train'
val_dir = '/home/data/imagenet/val'
data_config = resolve_data_config(vars(args), model=model_g, verbose=args.local_rank == 0)

INFO:root:Data processing configuration for current model + dataset:
INFO:root:	input_size: (3, 240, 240)
INFO:root:	interpolation: bicubic
INFO:root:	mean: (0.485, 0.456, 0.406)
INFO:root:	std: (0.229, 0.224, 0.225)
INFO:root:	crop_pct: 0.882


In [15]:
num_aug_splits = 0

In [16]:
if args.distributed:
    if args.sync_bn:
        assert not args.split_bn
        try:
            if has_apex:
                model = convert_syncbn_model(model)
            else:
                model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
            if args.local_rank == 0:
                logging.info(
                    'Converted model to use Synchronized BatchNorm. WARNING: You may have issues if using '
                    'zero initialized BN layers (enabled by default for ResNets) while sync-bn enabled.')
        except Exception as e:
            logging.error('Failed to enable Synchronized BatchNorm. Install Apex or Torch >= 1.1')
    if has_apex:
        model = DDP(model, delay_allreduce=True)
    else:
        if args.local_rank == 0:
            logging.info("Using torch DistributedDataParallel. Install NVIDIA Apex for Apex DDP.")
        model = DDP(model, device_ids=[args.local_rank])  # can use device str in Torch >= 1.1
    # NOTE: EMA model does not need to be wrapped by DDP

In [17]:
# train_dataset = Dataset(train_dir)
val_dataset = Dataset(val_dir, load_bytes=False, class_map='')

In [18]:
param_count = sum([m.numel() for m in model_g.parameters()])
logging.info('Model created, param count: %d' % (param_count))

INFO:root:Model created, param count: 3662851


In [19]:
collate_fn = None
if args.prefetcher and args.mixup > 0:
    assert not num_aug_splits  # collate conflict (need to support deinterleaving in collate mixup)
    collate_fn = FastCollateMixup(args.mixup, args.smoothing, args.num_classes)

In [20]:
model_raw, test_time_pool = apply_test_time_pool(model_raw, data_config, args)

In [21]:
transform = transforms_factory.transforms_imagenet_train(img_size=args.img_size, auto_augment=False, color_jitter=args.color_jitter, interpolation=args.interpolation, re_count=args.recount, re_mode=args.remode, re_prob=args.reprob)

In [22]:
train_loader = get_dataloader_sample(transform, dataset='imagenet', batch_size=args.batch_size, is_sample=True, k=4096)

stage1 finished!
dataset initialized!
num_samples 1281167
num_class 1000


In [23]:
crop_pct = 1.0 if test_time_pool else data_config['crop_pct']
val_loader = create_loader(
    val_dataset,
    input_size=data_config['input_size'],
    batch_size=args.batch_size,
    is_training=False,
    use_prefetcher=args.prefetcher,
    interpolation=data_config['interpolation'],
    mean=data_config['mean'],
    std=data_config['std'],
    num_workers=args.workers,
    crop_pct=crop_pct,
    pin_memory=args.pin_mem,
    tf_preprocessing=args.tf_preprocessing)

In [24]:
if args.jsd:
    assert num_aug_splits > 1  # JSD only valid with aug splits set
    train_loss_fn = JsdCrossEntropy(num_splits=num_aug_splits, smoothing=args.smoothing)
    validate_loss_fn = nn.CrossEntropyLoss()
elif args.mixup > 0.:
    # smoothing is handled with mixup label transform
    train_loss_fn = SoftTargetCrossEntropy()
    validate_loss_fn = nn.CrossEntropyLoss()
elif args.smoothing:
    train_loss_fn = LabelSmoothingCrossEntropy(smoothing=args.smoothing)
    validate_loss_fn = nn.CrossEntropyLoss()
else:
    train_loss_fn = nn.CrossEntropyLoss()
    validate_loss_fn = train_loss_fn
    
crd_loss_fn = CRDLoss(args).cuda()
critic_loss_fn = nn.BCELoss()
real_label = 1
fake_label = 0

In [25]:
trainable_list = nn.ModuleList([])
trainable_list.append(model_g)
# trainable_list.append(model_raw)
trainable_list.append(crd_loss_fn.embed_s)
trainable_list.append(crd_loss_fn.embed_t)

ModuleList(
  (0): Generator(
    (from_rgb): Conv2d(3, 68, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (encode): ModuleList(
      (0): ResBlk(
        (actv): LeakyReLU(negative_slope=0.2)
        (conv1): Conv2d(68, 68, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(68, 136, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm1): InstanceNorm2d(68, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (norm2): InstanceNorm2d(68, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (conv1x1): Conv2d(68, 136, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): ResBlk(
        (actv): LeakyReLU(negative_slope=0.2)
        (conv1): Conv2d(136, 136, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(136, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm1): InstanceNorm2d(136, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
  

In [26]:
optimizer = create_optimizer(args, trainable_list)
lr_scheduler, num_epochs = create_scheduler(args, optimizer)

In [27]:
optimizer_critic = create_optimizer(args, model_critic)
lr_scheduler_critic, _ = create_scheduler(args, optimizer_critic)

In [28]:
# optionally resume from a checkpoint
resume_state = {}
resume_epoch = None
if args.resume:
    resume_state, resume_epoch = resume_checkpoint(model_g, args.resume)
if resume_state and not args.no_resume_opt:
    if 'optimizer' in resume_state:
        if args.local_rank == 0:
            logging.info('Restoring Optimizer state from checkpoint')
        optimizer.load_state_dict(resume_state['optimizer'])
    if use_amp and 'amp' in resume_state and 'load_state_dict' in amp.__dict__:
        if args.local_rank == 0:
            logging.info('Restoring NVIDIA AMP state from checkpoint')
        amp.load_state_dict(resume_state['amp'])
del resume_state

In [29]:
model_g = torch.nn.DataParallel(model_g)

In [30]:
if args.local_rank == 0:
    logging.info('Scheduled epochs: {}'.format(num_epochs))

INFO:root:Scheduled epochs: 200


In [31]:
resume_state = {}
resume_epoch = None
start_epoch = 0
if args.start_epoch is not None:
    # a specified start_epoch will always override the resume epoch
    start_epoch = args.start_epoch
elif resume_epoch is not None:
    start_epoch = resume_epoch
if lr_scheduler is not None and start_epoch > 0:
    lr_scheduler.step(start_epoch)

In [32]:
eval_metric = args.eval_metric
best_metric = None
best_epoch = None
saver = None
output_dir = ''
if args.local_rank == 0:
    output_base = args.output if args.output else './output'
    exp_name = '-'.join([
        datetime.now().strftime("%Y%m%d-%H%M%S"),
        args.model,
        str(data_config['input_size'][-1])
    ])
    output_dir = get_outdir(output_base, 'train', exp_name)
    decreasing = True if eval_metric == 'loss' else False
    saver = CheckpointSaver(checkpoint_dir=output_dir, decreasing=decreasing)

In [33]:
model_ema = None
if args.model_ema:
    # Important to create EMA model after cuda(), DP wrapper, and AMP but before SyncBN and DDP wrapper
    model_ema = ModelEma(
        model,
        decay=args.model_ema_decay,
        device='cpu' if args.model_ema_force_cpu else '',
        resume=args.resume)

In [34]:
def train_epoch(epoch, model_g, model_raw, model_ns, model_critic, loader, optimizer, optimizer_critic, loss_fn, crd_loss_fn, critic_loss_fn, args,
               lr_scheduler=None, lr_scheduler_critic=None, saver=None, output_dir='', use_amp=False, model_ema=None):

    batch_time_m = AverageMeter()
    data_time_m = AverageMeter()
    losses_m = AverageMeter()
    losses_crd = AverageMeter()
    losses_ce = AverageMeter()
    losses_kd = AverageMeter()
    losses_critic = AverageMeter()
    losses_g = AverageMeter()
    
    model_g.train()
    model_critic.train()
    model_raw.eval()
    model_ns.eval()
    
    end = time.time()
    last_idx = len(loader) - 1
    num_updates = epoch * len(loader)
        
    for batch_idx, data in enumerate(loader):
        inputs, target, index, contrast_idx = data
        inputs, target, index, contrast_idx = inputs.cuda(), target.cuda(), index.cuda(), contrast_idx.cuda()
        last_batch = batch_idx == last_idx
        data_time_m.update(time.time() - end)
        
        g_out = model_g(inputs)
        
        inputs_z = inputs + g_out
        output, f_raw = model_raw(inputs_z)
        with torch.no_grad():
#             inputs_ns = F.interpolate(inputs, size=300, mode='bicubic')
            out_ns, f_ns = model_ns(inputs)
            out_ns = out_ns.detach()
        
        # Critic
        model_critic.train()
        model_critic.zero_grad()
        label = torch.full((f_raw.size(0), ), real_label, device='cuda')
        critic_out = model_critic(f_ns).view(-1)
        loss_critic_ns = args.lambda_critic * critic_loss_fn(critic_out, label)
        loss_critic_ns.backward()
        loss_critic_ns_out = loss_critic_ns.item()
        
        label.fill_(fake_label)
        critic_out = model_critic(f_raw.detach()).view(-1)
        loss_critic_raw = critic_loss_fn(critic_out, label)
        loss_critic_raw.backward()
        loss_critic_raw_out = args.lambda_critic * loss_critic_raw.item()
        loss_critic = loss_critic_ns_out + loss_critic_raw_out
        optimizer_critic.step()
        
        # KD
        p_s = F.log_softmax(output/args.T, dim=1)
        p_t = F.softmax(out_ns/args.T, dim=1)
        loss_kd = F.kl_div(p_s, p_t, size_average=False) * (args.T ** 2) / output.shape[0]
        
       # CE
        loss_ce = loss_fn(output, target)
        
        # CRD
        loss_crd = crd_loss_fn(f_raw, f_ns, index, contrast_idx)
        
        # G
        model_critic.eval()
        label.fill_(real_label)
        critic_g_out = model_critic(f_raw).view(-1)
        loss_g = critic_loss_fn(critic_g_out, label)
        
        # overall loss
        loss = args.lambda_kd * loss_kd + args.lambda_ce * loss_ce + args.lambda_crd * loss_crd + args.lambda_g * loss_g
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if not args.distributed:
            losses_kd.update(loss_kd.item(), inputs.size(0))
            losses_ce.update(loss_ce.item(), inputs.size(0))
            losses_crd.update(loss_crd.item(), inputs.size(0))
            losses_critic.update(loss_critic, inputs.size(0))
            losses_g.update(loss_g.item(), inputs.size(0))
            losses_m.update(loss.item(), inputs.size(0))

        if model_ema is not None:
            model_ema.update(model)
        num_updates += 1

        batch_time_m.update(time.time() - end)
        if last_batch or batch_idx % args.log_interval == 0:
            lrl = [param_group['lr'] for param_group in optimizer.param_groups]
            lr = sum(lrl) / len(lrl)

            if args.distributed:
                reduced_loss = reduce_tensor(loss.data, args.world_size)
                losses_m.update(reduced_loss.item(), inputs.size(0))

            if args.local_rank == 0:
                logging.info(
                    'Train: {} [{:>4d}/{} ({:>3.0f}%)]  '
                    'Loss: {loss.val:>9.6f} ({loss.avg:>6.4f})  '
                    'Loss_kd: {loss_kd.val:>9.6f} ({loss_kd.avg:>6.4f})  '
                    'Loss_ce: {loss_ce.val:>9.6f} ({loss_ce.avg:>6.4f})  '
                    'Loss_crd: {loss_crd.val:>9.6f} ({loss_crd.avg:>6.4f})  '
                    'Loss_g: {loss_g.val:>9.6f} ({loss_g.avg:>6.4f})  '
                    'Loss_critic: {loss_critic.val:>9.6f} ({loss_critic.avg:>6.4f})  '
                    'Time: {batch_time.val:.3f}s, {rate:>7.2f}/s  '
                    '({batch_time.avg:.3f}s, {rate_avg:>7.2f}/s)  '
                    'LR: {lr:.7f}  '
                    'Data: {data_time.val:.3f} ({data_time.avg:.3f})'.format(
                        epoch,
                        batch_idx, len(loader),
                        100. * batch_idx / last_idx,
                        loss=losses_m,
                        loss_kd=losses_kd,
                        loss_ce=losses_ce,
                        loss_crd=losses_crd,
                        loss_g=losses_g,
                        loss_critic=losses_critic,
                        batch_time=batch_time_m,
                        rate=inputs.size(0) * args.world_size / batch_time_m.val,
                        rate_avg=inputs.size(0) * args.world_size / batch_time_m.avg,
                        lr=lr,
                        data_time=data_time_m))

                if args.save_images and output_dir:
                    torchvision.utils.save_image(
                        inputs_z,
                        os.path.join(output_dir, 'train-batch-%d.jpg' % batch_idx),
                        padding=0,
                        normalize=True)

        if saver is not None and args.recovery_interval and (
                last_batch or (batch_idx + 1) % args.recovery_interval == 0):
            saver.save_recovery(
                model_raw, optimizer, args, epoch, model_ema=model_ema, use_amp=use_amp, batch_idx=batch_idx)
            
        if lr_scheduler is not None:
            lr_scheduler.step_update(num_updates=num_updates, metric=losses_m.avg)
            lr_scheduler_critic.step_update(num_updates=num_updates, metric=losses_critic.avg)

        end = time.time()
        # end for

    if hasattr(optimizer, 'sync_lookahead'):
        optimizer.sync_lookahead()
        

    return OrderedDict([('loss', losses_m.avg)])

In [35]:
def val_epoch(model_raw, model, val_loader, criterion, args):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    model_raw.eval()
    model.eval()
#     model_att.eval()
    with torch.no_grad():
        # warmup, reduce variability of first batch time, especially for comparing torchscript vs non
        end = time.time()
        for i, (inputs, target) in enumerate(val_loader):
            if args.no_prefetcher:
                target = target.cuda()
                inputs = inputs.cuda()
            
            out = model(inputs)
            # synthesizing input + generator
            inputs_out = inputs + out
            # compute output
            output, _ = model_raw(inputs_out)
#             output = model_att(inputs_out, output)
            loss = criterion(output, target)
            
            # measure accuracy and record loss
            acc1, acc5 = accuracy(output.data, target, topk=(1, 5))
            losses.update(loss.item(), inputs.size(0))
            top1.update(acc1.item(), inputs.size(0))
            top5.update(acc5.item(), inputs.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.log_freq == 0:
                logging.info(
                    'Test: [{0:>4d}/{1}]  '
                    'Time: {batch_time.val:.3f}s ({batch_time.avg:.3f}s, {rate_avg:>7.2f}/s)  '
                    'Loss: {loss.val:>7.4f} ({loss.avg:>6.4f})  '
                    'Acc@1: {top1.val:>7.3f} ({top1.avg:>7.3f})  '
                    'Acc@5: {top5.val:>7.3f} ({top5.avg:>7.3f})'.format(
                        i, len(val_loader), batch_time=batch_time,
                        rate_avg=inputs.size(0) / batch_time.avg,
                        loss=losses, top1=top1, top5=top5))

    results = OrderedDict(
        top1=round(top1.avg, 4), top1_err=round(100 - top1.avg, 4),
        top5=round(top5.avg, 4), top5_err=round(100 - top5.avg, 4),
        param_count=round(param_count / 1e6, 2),
        img_size=data_config['input_size'][-1],
        cropt_pct=crop_pct,
        interpolation=data_config['interpolation'])

    logging.info(' * Acc@1 {:.3f} ({:.3f}) Acc@5 {:.3f} ({:.3f})'.format(
       results['top1'], results['top1_err'], results['top5'], results['top5_err']))
    return results

In [ ]:
for epoch in range(start_epoch, num_epochs):
    if args.distributed:
        train_loader.sampler.set_epoch(epoch)

    train_metrics = train_epoch(
        epoch, model_g, model_raw, model_ns, model_critic, train_loader, optimizer, optimizer_critic, 
        train_loss_fn, crd_loss_fn, critic_loss_fn, args,
        lr_scheduler=lr_scheduler, lr_scheduler_critic=lr_scheduler_critic, 
        saver=saver, output_dir=output_dir,
        use_amp=use_amp, model_ema=model_ema)

    eval_metrics = val_epoch(model_raw, model_g, val_loader, validate_loss_fn, args)

    if model_ema is not None and not args.model_ema_force_cpu:
        if args.distributed and args.dist_bn in ('broadcast', 'reduce'):
            distribute_bn(model_ema, args.world_size, args.dist_bn == 'reduce')

    if lr_scheduler is not None:
        # step LR for next epoch
        lr_scheduler.step(epoch + 1, eval_metrics[eval_metric])
        
    update_summary( 
        epoch, train_metrics, eval_metrics, os.path.join(output_dir, 'summary.csv'),
        write_header=best_metric is None)

    if saver is not None:
    # save proper checkpoint with eval metric
        save_metric = eval_metrics[eval_metric]
        best_metric, best_epoch = saver.save_checkpoint(
            model_g, optimizer, args,
            epoch=epoch, model_ema=model_ema, metric=save_metric, use_amp=use_amp)
        torch.save(model_raw, 'raw')
        torch.save(model_critic, 'critic')
        torch.save(crd_loss_fn.embed_s, 'embed_s')
        torch.save(crd_loss_fn.embed_t, 'embed_t')

/opt/conda/conda-bld/pytorch_1587428398394/work/aten/src/ATen/native/TensorFactories.cpp:361: UserWarning: Deprecation warning: In a future PyTorch release torch.full will no longer return tensors of floating dtype by default. Instead, a bool fill_value will return a tensor of torch.bool dtype, and an integral fill_value will return a tensor of torch.long dtype. Set the optional `dtype` or `out` arguments to suppress this warning.
/home/cutz/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


normalization constant Z_v1 is set to 2838143.5
normalization constant Z_v2 is set to 2847929.0


INFO:root:Train: 0 [   0/8374 (  0%)]  Loss: 22.082279 (22.0823)  Loss_kd:  0.893180 (0.8932)  Loss_ce:  4.082958 (4.0830)  Loss_crd: 20.687515 (20.6875)  Loss_g:  0.695158 (0.6952)  Loss_critic:  1.180867 (1.1809)  Time: 23.046s,   19.92/s  (23.046s,   19.92/s)  LR: 0.0050000  Data: 4.704 (4.704)
INFO:root:Train: 0 [  50/8374 (  1%)]  Loss: 19.465551 (19.4820)  Loss_kd:  0.364290 (0.3818)  Loss_ce:  1.645583 (1.8191)  Loss_crd: 20.560423 (20.5064)  Loss_g:  1.259172 (1.0950)  Loss_critic:  0.759203 (0.9239)  Time: 1.204s,  381.37/s  (1.633s,  281.06/s)  LR: 0.0050000  Data: 0.013 (0.105)
INFO:root:Train: 0 [ 100/8374 (  1%)]  Loss: 19.707054 (19.4880)  Loss_kd:  0.360005 (0.3682)  Loss_ce:  1.816164 (1.7559)  Loss_crd: 20.474302 (20.4961)  Loss_g:  1.439304 (1.2088)  Loss_critic:  0.623973 (0.8020)  Time: 1.158s,  396.36/s  (1.406s,  326.50/s)  LR: 0.0050000  Data: 0.013 (0.059)
INFO:root:Train: 0 [ 150/8374 (  2%)]  Loss: 19.801529 (19.5516)  Loss_kd:  0.336105 (0.3634)  Loss_ce:  1.

INFO:root:Train: 0 [1400/8374 ( 17%)]  Loss: 20.682241 (20.3586)  Loss_kd:  0.370855 (0.3557)  Loss_ce:  1.739907 (1.7053)  Loss_crd: 19.847898 (20.3584)  Loss_g:  3.366451 (2.5137)  Loss_critic:  0.211618 (0.3427)  Time: 1.197s,  383.35/s  (1.186s,  386.92/s)  LR: 0.0050000  Data: 0.013 (0.016)
INFO:root:Train: 0 [1450/8374 ( 17%)]  Loss: 20.433147 (20.3732)  Loss_kd:  0.356634 (0.3557)  Loss_ce:  1.669726 (1.7047)  Loss_crd: 19.879255 (20.3522)  Loss_g:  3.129230 (2.5388)  Loss_critic:  0.235319 (0.3386)  Time: 1.158s,  396.37/s  (1.186s,  387.14/s)  LR: 0.0050000  Data: 0.013 (0.016)
INFO:root:Train: 0 [1500/8374 ( 18%)]  Loss: 20.769579 (20.3889)  Loss_kd:  0.341288 (0.3558)  Loss_ce:  1.731587 (1.7046)  Loss_crd: 19.924698 (20.3468)  Loss_g:  3.446181 (2.5638)  Loss_critic:  0.241473 (0.3347)  Time: 1.159s,  395.92/s  (1.185s,  387.34/s)  LR: 0.0050000  Data: 0.013 (0.016)
INFO:root:Train: 0 [1550/8374 ( 19%)]  Loss: 21.077749 (20.4044)  Loss_kd:  0.390110 (0.3558)  Loss_ce:  1.79

INFO:root:Train: 0 [2800/8374 ( 33%)]  Loss: 20.969709 (20.6486)  Loss_kd:  0.358335 (0.3579)  Loss_ce:  1.698305 (1.7072)  Loss_crd: 19.882210 (20.1913)  Loss_g:  3.759124 (3.0380)  Loss_critic:  0.140390 (0.2721)  Time: 1.174s,  391.06/s  (1.178s,  389.67/s)  LR: 0.0050000  Data: 0.013 (0.015)
INFO:root:Train: 0 [2850/8374 ( 34%)]  Loss: 20.737440 (20.6546)  Loss_kd:  0.363412 (0.3579)  Loss_ce:  1.598894 (1.7072)  Loss_crd: 19.792110 (20.1848)  Loss_g:  3.676805 (3.0520)  Loss_critic:  0.175033 (0.2707)  Time: 1.159s,  395.97/s  (1.178s,  389.70/s)  LR: 0.0050000  Data: 0.013 (0.015)
INFO:root:Train: 0 [2900/8374 ( 35%)]  Loss: 20.906832 (20.6607)  Loss_kd:  0.354328 (0.3580)  Loss_ce:  1.732720 (1.7073)  Loss_crd: 19.716816 (20.1787)  Loss_g:  3.807915 (3.0655)  Loss_critic:  0.187338 (0.2692)  Time: 1.161s,  395.38/s  (1.178s,  389.74/s)  LR: 0.0050000  Data: 0.013 (0.015)
INFO:root:Train: 0 [2950/8374 ( 35%)]  Loss: 20.714069 (20.6659)  Loss_kd:  0.363354 (0.3581)  Loss_ce:  1.68

...

INFO:root:Test: [  90/327]  Time: 0.335s (0.417s,  366.94/s)  Loss:  0.8182 (0.8619)  Acc@1:  79.085 ( 79.516)  Acc@5:  96.732 ( 95.037)
INFO:root:Test: [ 100/327]  Time: 0.333s (0.409s,  374.49/s)  Loss:  0.4980 (0.8553)  Acc@1:  89.542 ( 79.641)  Acc@5:  96.732 ( 95.166)
INFO:root:Test: [ 110/327]  Time: 0.331s (0.402s,  380.83/s)  Loss:  0.7202 (0.8430)  Acc@1:  84.967 ( 80.045)  Acc@5:  96.732 ( 95.225)
INFO:root:Test: [ 120/327]  Time: 0.333s (0.396s,  386.05/s)  Loss:  0.8917 (0.8408)  Acc@1:  81.699 ( 80.046)  Acc@5:  95.425 ( 95.257)
INFO:root:Test: [ 130/327]  Time: 0.332s (0.391s,  390.82/s)  Loss:  1.3716 (0.8519)  Acc@1:  66.667 ( 79.734)  Acc@5:  90.196 ( 95.101)
INFO:root:Test: [ 140/327]  Time: 0.335s (0.387s,  394.92/s)  Loss:  0.9549 (0.8812)  Acc@1:  78.431 ( 79.080)  Acc@5:  92.157 ( 94.785)
INFO:root:Test: [ 150/327]  Time: 0.331s (0.384s,  398.65/s)  Loss:  1.8785 (0.9167)  Acc@1:  52.941 ( 78.280)  Acc@5:  84.967 ( 94.408)
INFO:root:Test: [ 160/327]  Time: 0.337s 

INFO:root:Train: 8 [ 700/8374 (  8%)]  Loss:  8.859299 (8.6563)  Loss_kd:  0.423995 (0.4287)  Loss_ce:  2.013326 (1.9719)  Loss_crd:  4.018579 (3.8325)  Loss_g:  4.008893 (3.9872)  Loss_critic:  0.316768 (0.3523)  Time: 1.176s,  390.38/s  (1.191s,  385.29/s)  LR: 0.0045634  Data: 0.013 (0.020)
INFO:root:Train: 8 [ 750/8374 (  9%)]  Loss:  8.687989 (8.6548)  Loss_kd:  0.404443 (0.4284)  Loss_ce:  1.984691 (1.9715)  Loss_crd:  3.702508 (3.8321)  Loss_g:  4.171061 (3.9865)  Loss_critic:  0.409229 (0.3522)  Time: 1.176s,  390.45/s  (1.191s,  385.49/s)  LR: 0.0045634  Data: 0.013 (0.019)
INFO:root:Train: 8 [ 800/8374 ( 10%)]  Loss:  8.819963 (8.6566)  Loss_kd:  0.421920 (0.4282)  Loss_ce:  2.022033 (1.9718)  Loss_crd:  3.860981 (3.8323)  Loss_g:  4.109030 (3.9884)  Loss_critic:  0.359429 (0.3512)  Time: 1.175s,  390.79/s  (1.190s,  385.77/s)  LR: 0.0045634  Data: 0.013 (0.019)
INFO:root:Train: 8 [ 850/8374 ( 10%)]  Loss:  8.722409 (8.6522)  Loss_kd:  0.408358 (0.4280)  Loss_ce:  2.066408 (1

INFO:root:Train: 8 [2100/8374 ( 25%)]  Loss:  8.401746 (8.6526)  Loss_kd:  0.422319 (0.4283)  Loss_ce:  1.983679 (1.9705)  Loss_crd:  3.721817 (3.8301)  Loss_g:  3.772867 (3.9872)  Loss_critic:  0.365119 (0.3523)  Time: 1.187s,  386.61/s  (1.186s,  387.12/s)  LR: 0.0045634  Data: 0.013 (0.015)
INFO:root:Train: 8 [2150/8374 ( 26%)]  Loss:  8.665565 (8.6517)  Loss_kd:  0.439622 (0.4283)  Loss_ce:  1.970441 (1.9707)  Loss_crd:  3.912959 (3.8297)  Loss_g:  3.906417 (3.9861)  Loss_critic:  0.378131 (0.3525)  Time: 1.173s,  391.15/s  (1.186s,  387.15/s)  LR: 0.0045634  Data: 0.013 (0.015)
INFO:root:Train: 8 [2200/8374 ( 26%)]  Loss:  8.813022 (8.6520)  Loss_kd:  0.450336 (0.4284)  Loss_ce:  2.035244 (1.9710)  Loss_crd:  3.975766 (3.8304)  Loss_g:  3.933536 (3.9854)  Loss_critic:  0.338890 (0.3526)  Time: 1.182s,  388.43/s  (1.185s,  387.18/s)  LR: 0.0045634  Data: 0.013 (0.015)
INFO:root:Train: 8 [2250/8374 ( 27%)]  Loss:  9.180508 (8.6505)  Loss_kd:  0.418070 (0.4283)  Loss_ce:  2.088661 (1

INFO:root:Train: 8 [3500/8374 ( 42%)]  Loss:  8.798649 (8.6452)  Loss_kd:  0.431510 (0.4282)  Loss_ce:  2.041037 (1.9712)  Loss_crd:  3.948641 (3.8295)  Loss_g:  3.958986 (3.9778)  Loss_critic:  0.383165 (0.3534)  Time: 1.173s,  391.42/s  (1.185s,  387.42/s)  LR: 0.0045634  Data: 0.013 (0.014)
INFO:root:Train: 8 [3550/8374 ( 42%)]  Loss:  8.991989 (8.6450)  Loss_kd:  0.433930 (0.4282)  Loss_ce:  2.021553 (1.9711)  Loss_crd:  3.898734 (3.8294)  Loss_g:  4.271898 (3.9776)  Loss_critic:  0.389760 (0.3534)  Time: 1.173s,  391.25/s  (1.185s,  387.43/s)  LR: 0.0045634  Data: 0.013 (0.014)
INFO:root:Train: 8 [3600/8374 ( 43%)]  Loss:  8.134338 (8.6441)  Loss_kd:  0.419814 (0.4282)  Loss_ce:  1.892596 (1.9711)  Loss_crd:  3.556051 (3.8294)  Loss_g:  3.721358 (3.9766)  Loss_critic:  0.433602 (0.3535)  Time: 1.177s,  389.99/s  (1.185s,  387.42/s)  LR: 0.0045634  Data: 0.013 (0.014)
INFO:root:Train: 8 [3650/8374 ( 44%)]  Loss:  8.162527 (8.6438)  Loss_kd:  0.400847 (0.4283)  Loss_ce:  1.787676 (1

INFO:root:Train: 8 [4900/8374 ( 59%)]  Loss:  8.001345 (8.6357)  Loss_kd:  0.378006 (0.4283)  Loss_ce:  1.871968 (1.9710)  Loss_crd:  3.564718 (3.8294)  Loss_g:  3.624496 (3.9659)  Loss_critic:  0.290570 (0.3544)  Time: 1.183s,  387.99/s  (1.185s,  387.47/s)  LR: 0.0045634  Data: 0.013 (0.014)
INFO:root:Train: 8 [4950/8374 ( 59%)]  Loss:  9.088303 (8.6350)  Loss_kd:  0.451869 (0.4283)  Loss_ce:  2.073936 (1.9710)  Loss_crd:  3.987571 (3.8292)  Loss_g:  4.215550 (3.9654)  Loss_critic:  0.367728 (0.3544)  Time: 1.174s,  390.94/s  (1.185s,  387.47/s)  LR: 0.0045634  Data: 0.013 (0.014)
INFO:root:Train: 8 [5000/8374 ( 60%)]  Loss:  8.702408 (8.6344)  Loss_kd:  0.428741 (0.4283)  Loss_ce:  1.943414 (1.9710)  Loss_crd:  3.926168 (3.8290)  Loss_g:  3.986646 (3.9648)  Loss_critic:  0.399999 (0.3546)  Time: 1.189s,  386.13/s  (1.185s,  387.47/s)  LR: 0.0045634  Data: 0.013 (0.014)
INFO:root:Train: 8 [5050/8374 ( 60%)]  Loss:  8.491601 (8.6339)  Loss_kd:  0.440062 (0.4283)  Loss_ce:  1.796407 (1